In [ ]:
"""
WeatherSense App
Created by: Keith Neomi Cruz, Jealn Maelryn Gregorio, Leon Alphonsus Binarao
"""

import requests
import tkinter as tk
from tkinter import messagebox
import matplotlib.pyplot as plt
from datetime import datetime


# ------------------------------------------------------------
# WEATHER EMOJI DICTIONARY
# ------------------------------------------------------------
weather_emojis = {
    0: "☀️ Clear sky",
    1: "🌤️ Mostly clear",
    2: "⛅ Partly cloudy",
    3: "☁️ Cloudy",
    45: "🌫️ Fog",
    48: "🌫️ Depositing rime fog",
    51: "🌦️ Light drizzle",
    53: "🌦️ Moderate drizzle",
    55: "🌧️ Heavy drizzle",
    56: "🌧️ Light freezing drizzle",
    57: "🌧️ Heavy freezing drizzle",
    61: "🌦️ Light rain",
    63: "🌧️ Moderate rain",
    65: "🌧️ Heavy rain",
    71: "🌨️ Light snow",
    73: "🌨️ Moderate snow",
    75: "❄️ Heavy snow",
    77: "🌨️ Snow grains",
    80: "🌦️ Rain showers",
    81: "🌧️ Rain showers",
    82: "⛈️ Heavy rain showers",
    85: "🌨️ Snow showers",
    86: "❄️ Heavy snow showers",
    95: "⛈️ Thunderstorm",
    96: "⛈️ Thunderstorm w/ light hail",
    99: "⛈️ Thunderstorm w/ heavy hail"
}


# ------------------------------------------------------------
# GEOCODER – WORKS FOR *ALL CITIES*
# ------------------------------------------------------------
def get_coordinates(city_name):
    url = f"https://geocoding-api.open-meteo.com/v1/search?name={city_name}&count=1"

    try:
        response = requests.get(url).json()

        if "results" not in response:
            return None

        result = response["results"][0]
        return result["latitude"], result["longitude"], result["name"], result["country"]

    except:
        return None


# ------------------------------------------------------------
# GET CURRENT WEATHER
# ------------------------------------------------------------
def get_current_weather():
    city = city_entry.get().strip()

    if city == "":
        messagebox.showerror("Error", "Please enter a city name.")
        return

    coords = get_coordinates(city)

    if not coords:
        messagebox.showerror("Not Found", "City not found. Try another name.")
        return

    lat, lon, realname, country = coords

    url = (
        "https://api.open-meteo.com/v1/forecast?"
        f"latitude={lat}&longitude={lon}&current_weather=true&timezone=auto"
    )

    try:
        data = requests.get(url).json()
        current = data["current_weather"]

        temp = current.get("temperature")
        wind = current.get("windspeed")
        code = current.get("weathercode")

        # Get emoji label
        emoji_text = weather_emojis.get(code, "❓ Unknown weather")

        # Update GUI
        location_label.config(text=f"{realname}, {country}")
        temp_label.config(text=f"Temperature: {temp}°C")
        wind_label.config(text=f"Wind Speed: {wind} km/h")
        emoji_label.config(text=f"Weather: {emoji_text}")

        # Save for forecast
        global last_lat, last_lon
        last_lat, last_lon = lat, lon

    except:
        messagebox.showerror("Error", "Unable to retrieve weather.")


# ------------------------------------------------------------
# FORECAST GRAPH
# ------------------------------------------------------------
def show_forecast_chart():
    try:
        lat = last_lat
        lon = last_lon
    except:
        messagebox.showwarning("Error", "Get weather first.")
        return

    url = (
        "https://api.open-meteo.com/v1/forecast?"
        f"latitude={lat}&longitude={lon}"
        "&hourly=temperature_2m,relativehumidity_2m,weathercode"
        "&timezone=auto"
    )

    try:
        data = requests.get(url).json()
        hourly = data["hourly"]

        times = hourly["time"]
        temps = hourly["temperature_2m"]
        hums = hourly["relativehumidity_2m"]

        # Convert time string → datetime
        dt_times = [datetime.fromisoformat(t) for t in times]

        plt.figure(figsize=(12, 5))
        plt.plot(dt_times, temps, label="Temperature (°C)")
        plt.plot(dt_times, hums, label="Humidity (%)")
        plt.xlabel("Date & Time")
        plt.ylabel("Values")
        plt.title("5-Day 3-Hour Forecast")
        plt.legend()
        plt.grid()
        plt.tight_layout()
        plt.show()

    except:
        messagebox.showerror("Error", "Unable to load forecast data.")


# ------------------------------------------------------------
# TKINTER GUI
# ------------------------------------------------------------
root = tk.Tk()
root.title("WeatherSense App")
root.geometry("420x330")
root.resizable(False, False)

title_label = tk.Label(root, text="WeatherSense App", font=("Arial", 16, "bold"))
title_label.pack(pady=8)

authors_label = tk.Label(root, text="Created by: Keith Neomi Cruz | Jealn Maelryn Gregorio | Leon Alphonsus Binarao", font=("Arial", 7, "italic"))
authors_label.pack(pady=(0, 8))

city_entry = tk.Entry(root, width=30)
city_entry.pack()
city_entry.insert(0, "Enter city name...")

btn = tk.Button(root, text="Get Weather", command=get_current_weather)
btn.pack(pady=5)

# Display area
location_label = tk.Label(root, text="--", font=("Arial", 12, "bold", "italic"))
location_label.pack()

temp_label = tk.Label(root, text="Temperature: --")
temp_label.pack()

wind_label = tk.Label(root, text="Wind Speed: --")
wind_label.pack()

emoji_label = tk.Label(root, text="Weather: --")
emoji_label.pack()

forecast_btn = tk.Button(root, text="Show Forecast Chart", command=show_forecast_chart)
forecast_btn.pack(pady=15)

root.mainloop()
